In [1]:
from utils import Utils 
import pandas as pd
utils=Utils()

# BASE PLAYER_SCRAPER

In [2]:
df = utils.cargar_excel("Mundial_Historic_players.xlsx")

In [4]:
df

,pais,Nombre_jugador,Posicion,Edad,Partidos_jugados,Club,Año,Campeón,pais_AND_Campeón,Continente
0,Inglaterra,Gordon Banks,Portero,28,27,Leicester City,1966,Inglaterra,1,Europa
1,Inglaterra,George Cohen,Defensa,26,24,Fulham,1966,Inglaterra,1,Europa
2,Inglaterra,Ray Wilson,Defensa,31,45,Everton,1966,Inglaterra,1,Europa
3,Inglaterra,Nobby Stiles,Centrocampista,24,14,Manchester United,1966,Inglaterra,1,Europa
4,Inglaterra,Jack Charlton,Defensa,31,16,Leeds United,1966,Inglaterra,1,Europa
...,...,...,...,...,...,...,...,...,...,...
6270,Senegal,M'Baye Niang,Delantero,23,11,Torino,2018,Francia,0,África
6271,Senegal,Keita Baldé,Delantero,23,19,Mónaco,2018,Francia,0,África
6272,Senegal,Lamine Gassama,Defensa,28,40,Alanyaspor,2018,Francia,0,África
6273,Senegal,Moussa Wagué,Defensa,19,10,Eupen,2018,Francia,0,África


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from bs4 import BeautifulSoup

def buscar_wikipedia_jugadores(df):
    """
    Busca en Google los nombres de los jugadores y obtiene su enlace de Wikipedia si existe.

    Parámetros:
    df (pd.DataFrame): DataFrame con la columna 'Nombre_jugador'.

    Retorna:
    pd.DataFrame: DataFrame con 'Nombre_jugador' y 'wiki_link'.
    """
    # Configuración del WebDriver
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)

    resultados = []  # Lista para almacenar los resultados

    for nombre in df["Nombre_jugador"]:
        try:
            # Buscar en Google
            query = f"{nombre} site:wikipedia.org"
            url_busqueda = f"https://www.duckduckgo.com/?q={query.replace(' ', '+')}"
            driver.get(url_busqueda)
            
            # Esperar que aparezcan los resultados de búsqueda
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # Buscar el primer enlace de Wikipedia en los resultados
            link_wiki = None
            for enlace in soup.find_all("a", href=True):
                href = enlace["href"]
                if "wikipedia.org" in href:
                    link_wiki = href
                    break

            # Limpiar el enlace si tiene parámetros extra
            if link_wiki and "/url?q=" in link_wiki:
                link_wiki = link_wiki.split("/url?q=")[1].split("&")[0]

            # Agregar al resultado
            resultados.append({"Nombre_jugador": nombre, "wiki_link": link_wiki if link_wiki else "No encontrado"})

        except Exception as e:
            print(f"Error con {nombre}: {e}")
            resultados.append({"Nombre_jugador": nombre, "wiki_link": "Error"})

    # Cerrar el navegador
    driver.quit()

    # Crear DataFrame con los resultados
    df_links_names = pd.DataFrame(resultados)

    return df_links_names
